In [1]:
import unicodedata
import re
import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

import pandas as pd
import os
import time
import csv
import json
from typing import Dict, List, Optional, Union, cast
import requests
import prepare

from bs4 import BeautifulSoup

In [2]:
df = prepare.wrangle_data()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 290 entries, 0 to 391
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             290 non-null    object
 1   language         290 non-null    object
 2   readme_contents  290 non-null    object
 3   stemmed          290 non-null    object
 4   lemmatized       290 non-null    object
dtypes: object(5)
memory usage: 13.6+ KB


In [3]:
df.head()

,repo,language,readme_contents,stemmed,lemmatized
0,dreamingechoes/awesome-mental-health,HTML,"\n<p align=""center""><img src=""./media/logo.png...",aboutsparkl curat list awesom articl websit re...,aboutsparkles curated list awesome article web...
1,Flaque/quirk,TypeScript,🚧🚧🚧\n\n**Quirk is no longer being maintained.*...,quirk longer maintainedquirk start littl thing...,quirk longer maintainedquirk started little th...
4,sepandhaghighi/nafas,Python,"<div align=""center"">\n<img src=""https://github...",nafa &#9; tabl content overview instal usag is...,nafas &#9; table content overview installation...
5,youarerad/youareradweb,TypeScript,"<div align=""center""><img src=""https://res.clou...",welcom rise disord ' websit repow nonprofit co...,welcome rise disorder ' website repowe nonprof...
7,OSMIHelp/osmi-survey-graph,PHP,# 2016 OSMI Survey Graph\n\n## Installing and ...,2016 osmi survey graph instal run copi envexam...,2016 osmi survey graph installing running copy...


In [ ]:
# import pandas as pd
# import os
# import time
# import csv
# import json
# from typing import Dict, List, Optional, Union, cast
# import requests

# from bs4 import BeautifulSoup

# from env import github_token, github_username

# # Check if repo csv exists
# #if not os.path.isfile("repo.csv"):
    
    
# lang_list = ['JavaScript', 'HTML', 'Java', 'Python', 'CSS', 'C#', 'PHP', 'TypeScript', 'R']
# #     page_num = [1-23]
# repos = []

# #     for page in page_num:
# for i in range(1,41):

# #             url = 'https://github.com/search?l={lang}&p={i}&q=mental+health&type=repositories'
#     url = f"https://github.com/search?p={i}&q=%23mental-health&type=Repositories"
#     while True:
#         response = requests.get(url)
#         if response.ok:
#             print(True)
#             break
#         else:
#             print('sleeping')
#             time.sleep(20)
            
#     soup = BeautifulSoup(response.content, 'html.parser')

#     for element in soup.find_all('a', class_='v-align-middle'):
#         repos.append(element.text)

# #         time.sleep(10)

# with open('repo.csv', 'w') as createfile:
#     wr = csv.writer(createfile, quoting=csv.QUOTE_ALL)
#     wr.writerow(repos)
#     results = []
# with open('repo.csv', newline='') as inputfile:
#     results = list(csv.reader(inputfile))

#     REPOS = [item for sublist in results for item in sublist]
#     headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

# if headers["Authorization"] == "token " or headers["User-Agent"] == "":
#     raise Exception(
#     "You need to follow the instructions marked TODO in this script before trying to use it"
# )


# def github_api_request(url: str) -> Union[List, Dict]:
#     response = requests.get(url, headers=headers)
#     response_data = response.json()
#     if response.status_code != 200:
#         raise Exception(
#             f"Error response from github api! status code: {response.status_code}, "
#             f"response: {json.dumps(response_data)}"
#         )
#     return response_data


# def get_repo_language(repo: str) -> str:
#     url = f"https://api.github.com/repos/{repo}"
#     repo_info = github_api_request(url)
#     if type(repo_info) is dict:
#         repo_info = cast(Dict, repo_info)
#         return repo_info.get("language", None)
#     raise Exception(
#         f"Expecting a dictionary response from {url}, instead got {json.dumps(repo_info)}"
#     )
    
# def get_repo_contents(repo: str) -> List[Dict[str, str]]:
#     url = f"https://api.github.com/repos/{repo}/contents/"
#     contents = github_api_request(url)
#     if type(contents) is list:
#         contents = cast(List, contents)
#         return contents
#     raise Exception(
#         f"Expecting a list response from {url}, instead got {json.dumps(contents)}"
#     )

# def get_readme_download_url(files: List[Dict[str, str]]) -> str:
#     """
#     Takes in a response from the github api that lists the files in a repo and
#     returns the url that can be used to download the repo's README file.
#     """
#     for file in files:
#         if file["name"].lower().startswith("readme"):
#             return file["download_url"]
#     return ""

# def process_repo(repo: str) -> Dict[str, str]:
#     """
#     Takes a repo name like "gocodeup/codeup-setup-script" and returns a
#     dictionary with the language of the repo and the readme contents.
#     """
#     contents = get_repo_contents(repo)
#     readme_download_url = get_readme_download_url(contents)
#     if readme_download_url == "":
#         readme_contents = None
#     else:
#         readme_contents = requests.get(readme_download_url).text
#     return {
#         "repo": repo,
#         "language": get_repo_language(repo),
#         "readme_contents": readme_contents,
#     }

# def scrape_github_data() -> List[Dict[str, str]]:
#     """
#     Loop through all of the repos and process them. Returns the processed data.
#     """
#     return [process_repo(repo) for repo in REPOS]


# #if __name__ == "__main__":
# data = scrape_github_data()
# json.dump(data, open("data.json", "w"), indent=1)

In [ ]:
df = pd.read_json('data.json')

In [ ]:
df.info()

In [ ]:
df = df.dropna()
df = df[df['language'].map(df['language'].value_counts()) >= 5]

In [ ]:
df.language.value_counts()

In [ ]:
df = df.dropna()

In [ ]:
df.info()

In [ ]:
df['readme_original'] = df['readme_contents']

In [ ]:
def basic_clean(string):
    string = string.lower()
    string = (unicodedata.normalize('NFKD', string)
                         .encode('ascii', 'ignore')
                         .decode('utf-8', 'ignore')
             )
    string = re.sub(r"[^a-z0-9'\s]", '', string)
    return string

In [ ]:
df['readme_contents'] = df['readme_contents'].apply(basic_clean)

In [ ]:
df.head()

In [ ]:
def clean_html(string):
    string = re.sub(r'<[^>]*>', '', string)
    string = re.sub(r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)", '', string)
    string = re.sub(r'\n', '', string)
    string = re.sub(r'\s\s', '', string)
    return string

In [ ]:
df['readme_contents'] = df['readme_contents'].apply(clean_html)

In [ ]:
df.head()

In [ ]:
def tokenize(string):
    tokenizer = nltk.tokenize.ToktokTokenizer()
    return tokenizer.tokenize(string, return_str=True)

In [ ]:
df['readme_contents'] = df['readme_contents'].apply(tokenize)

In [ ]:
def stem(string):
    ps = nltk.porter.PorterStemmer()
    stems = [ps.stem(word) for word in string.split()]
    return ' '.join(stems)

In [ ]:
df['readme_contents'] = df['readme_contents'].apply(stem)

In [ ]:
df['readme_contents']

In [ ]:
def lemmatize(string):
    wnl = nltk.stem.WordNetLemmatizer()
    lemmas = [wnl.lemmatize(word) for word in string.split()]
    return ' '.join(lemmas)

In [ ]:
df['readme_contents'] = df['readme_contents'].apply(lemmatize)

In [ ]:
def remove_stopwords(string, extra_words=[], exclude_words=[]):
    stopword_list = stopwords.words('english')
    
    for word in extra_words:
        stopword_list.append(word)
    
    for word in exclude_words:
        stopword_list.remove(word)
        
    words = string.split()
    filtered_words = [word for word in words if word not in stopword_list]
    return ' '.join(filtered_words)

In [ ]:
df['readme_contents'] = df['readme_contents'].apply(remove_stopwords)

In [ ]:
df['readme_contents']

In [ ]:


def prepare_readme_data(df, column):
    clean_tokens = (df[column].apply(clean_html)
                              .apply(basic_clean)
                              .apply(tokenize)
                              .apply(remove_stopwords)
                   )
    
    for token in clean_tokens:
        token = ' '.join(token).split()
    
    df['stemmed'] = clean_tokens.apply(stem)
    df['lemmatized'] = clean_tokens.apply(lemmatize)
    return df

def wrangle_data():
    data = pd.read_json('data.json')
    return prepare_readme_data(data, 'readme_contents')